In [1]:
import pandas as pd

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import auc, roc_curve, roc_curve, roc_auc_score, classification_report, confusion_matrix, accuracy_score 

import matplotlib.pyplot as plt

import os 

import numpy as np

os.chdir('/home/jlm217/rds/rds-mrc-bsu-csoP2nj6Y6Y/mimah/stopah/lili')

from treedata import *

import missingno as msno


import numpy as np
import matplotlib.pyplot as plt
import sage 
import shap

import xgboost as xgb

R[write to console]: Warning:
R[write to console]:  namespace ‘pROC’ is not available and has been replaced
by .GlobalEnv when processing object ‘roc.objs’

/home/jlm217/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Missing-in-Attributes (MIA) approach 

The following is based on the CART split 

We will try the Missing-in-Attributes (MIA) approach 

Code missing as a separate category.


In [2]:
#Make all those values NA 

for x in stopah.columns:
    if len(stopah[stopah[x] == -2147483648].index) >0:
        #print(x)
        #print(stopah[stopah[x] == -2147483648].index)
        stopah[x][stopah[x] ==-2147483648] = np.nan
        
#stopah['Max.grams.of.alcohol.drunk.per.day..calc.'].loc[ [64, 164, 217, 220, 341, 351, 355, 507, 525, 538, 556, 599, 687, 693,
  #     723, 839, 933, 964, 986, 987, 991]]


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [3]:
stopah = stopah.drop(['D90_DTH'],axis=1) #Look at 28 day only 

old_col = list(stopah.columns)

In [4]:
mis_cat = []

categorical = ['D28_DTH','Prednisolone','Gender','Baseline_sepsis',
               'Baseline_GIB','PNPLA3_Rec','PNPLA3_Add','Hepatic.Encephalopathy...Merged']

for cat in categorical:
    print(cat +' : '+ str(stopah[cat].isna().sum()))
    if stopah[cat].isna().sum() > 0:
        mis_cat.append(cat)
        
#mis_cat

D28_DTH : 0
Prednisolone : 0
Gender : 0
Baseline_sepsis : 0
Baseline_GIB : 0
PNPLA3_Rec : 224
PNPLA3_Add : 224
Hepatic.Encephalopathy...Merged : 48


In [5]:
missing_cat1 = list(stopah[stopah['PNPLA3_Rec'].isna() == True].index)

In [6]:
stopah['PNPLA3_Rec_MIA'] = stopah['PNPLA3_Rec'].apply(lambda x: x==1 if x ==np.nan else 0)

stopah['PNPLA3_Add_MIA'] = stopah['PNPLA3_Add'].apply(lambda x: x==1 if x ==np.nan else 0)

In [7]:
stopah.drop(['PNPLA3_Add'], axis=1, inplace=True)

In [8]:
stopah['PNPLA3_Rec'].fillna(0,inplace=True)

In [9]:
#numerical variables 

missings = []

for x in stopah.columns:
    if (stopah[x].isna().sum())/len(stopah) > 0.0:
        missings.append(x)    

for mia in missings:
    mia_col1= mia + '1'
    mia_col2= mia + '2'
    
    stopah[mia_col1] =stopah[mia].fillna(-2147483648)
    stopah[mia_col2] =stopah[mia].fillna(2147483648)
    

In [10]:
#not sure about 'PNPLA3_Add', 'PNPLA3_Rec', HPCT_NG, Hepatic.Encephalopathy...Merged

#categoricals and targets 

excluded = ['D28_DTH','D90_DTH','Prednisolone', 'Hepatic.Encephalopathy...Merged',
                                            'Baseline_sepsis','Baseline_GIB','Gender', 'HPCT_NG',
                                            'PNPLA3_Add', 'PNPLA3_Rec']

In [11]:
stopah.drop(missings, axis=1, inplace=True) #same for 'bDNA'

In [12]:
df = stopah

In [13]:
df0 = df[df['Prednisolone']==0].drop(['Prednisolone'],axis=1)

df = df[df['Prednisolone']==1].drop(['Prednisolone'],axis=1)

X, y = df.drop('D28_DTH', axis=1), df[['D28_DTH']]

X0, y0 = df0.drop('D28_DTH', axis=1), df0[['D28_DTH']]

In [14]:

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X0_train, X0_test, y0_train, y0_test = train_test_split(X0, y0, random_state=1)

In [15]:
#Train

#Treated

clf = GradientBoostingClassifier(n_estimators=1000, 
                                 learning_rate=0.1,
                                 max_depth=6, 
                                 random_state=0).fit(X_train, y_train)

#Untreated

clf0= GradientBoostingClassifier(n_estimators=1000, 
                                 learning_rate=0.1,
                                 max_depth=6, 
                                 random_state=0).fit(X0_train, y0_train)


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


### Untreated vs treated training and test performance

In [16]:
print('Untreated')
print()
print('XGB Score:'+str(clf0.score(X0_test, y0_test)))

y0_predict = clf0.predict_proba(X0_test)

auc_score0 = roc_auc_score(y0_test, y0_predict[:,1])
print('AUC Score:',(auc_score0)*100)

print()
print('Treated')
print()

print('XGB Score:'+str(clf.score(X_test, y_test)))

y_predict = clf.predict_proba(X_test)

auc_score = roc_auc_score(y_test, y_predict[:,1])
print('AUC Score:',(auc_score)*100)

Untreated

XGB Score:0.8134328358208955
AUC Score: 75.71484528006268

Treated

XGB Score:0.8283582089552238
AUC Score: 81.10091743119266
